In [1]:
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from collections import Counter
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from dotenv import dotenv_values
config = dotenv_values('.env')

In [33]:
class Instaminer():
    def __init__(self, main_hashtag, max_hashtag):
        self.main_hashtag = main_hashtag 
        self.max_hashtag = max_hashtag
        self.browser = webdriver.Chrome(ChromeDriverManager().install())
        self.post_count_dic = {}
        self.insta_ID = config['insta_ID']
        self.insta_PD = config['insta_PD']
        
    def wait_for(self, locator):
        return WebDriverWait(self.browser, 4).until(
        EC.presence_of_element_located(locator)
        )
    
    def login(self):
        self.browser.get(f'https://www.instagram.com/accounts/login/')

        self.wait_for((By.CLASS_NAME, "_2hvTZ"))

        insta_id = self.browser.find_element_by_name('username')
        insta_pw = self.browser.find_element_by_name('password')

        insta_id.send_keys(self.insta_ID)
        insta_pw.send_keys(self.insta_PD)
        insta_pw.send_keys(Keys.ENTER)

        time.sleep(3)
    
    def save_file(self):
        with open(f"{self.main_hashtag}-report.csv", "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(['Hashtag', 'Post Count'])
            for k,v in self.post_count_dic.items():
                writer.writerow([k,v])
            
    def extract_data(self):
        try:
            hashtag = self.wait_for((By.CLASS_NAME, 'fKFbl')).text[1:]
            posts_count = int(self.wait_for((By.CLASS_NAME, 'g47SY')).text.replace(',',''))
            if hashtag and posts_count:
                self.post_count_dic[hashtag] = posts_count
                print(len(self.post_count_dic))
        except:
            pass
        
    def poppost2tag(self, frequency):
        pop_posts = self.browser.find_elements_by_class_name('_9AhH0')[:9]
        pop_posts_tags = []

        for post in pop_posts:
            post.click()
            try:
                self.wait_for((By.CLASS_NAME, "xil3i"))
                tag_list = self.browser.find_elements_by_class_name('xil3i')
                for tag in tag_list:
                    pop_posts_tags.append(tag.text[1:])
            except:
                pass
            ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
        ## 인기게시물들의 태그들 중 빈도수 순 10개 가져오기
        pop_posts_tags2 = [i for i in pop_posts_tags if i.encode().isalpha()]
        pop_tags_cnt = Counter(pop_posts_tags2).most_common(frequency)
        pop_tags = [i for i,c in pop_tags_cnt]
        if self.main_hashtag in pop_tags:
            pop_tags.remove(self.main_hashtag)
        return pop_tags
    
    def get_related(self, search_url):            
        self.browser.get(search_url)
        time.sleep(1)
        pop_tag_li = self.poppost2tag(15)

        ## 많이 나온 태그들 다시 검색해서 새탭에서 열기
        for tag in pop_tag_li:
            if tag not in self.post_count_dic.keys():
                search_url = f'https://www.instagram.com/explore/tags/{tag}'
                self.browser.execute_script("window.open('" +search_url +"');")
        for i in self.browser.window_handles:
            self.browser.switch_to.window(i)    
            self.extract_data()
            time.sleep(1)
        for window in self.browser.window_handles[0:-1]:
            self.browser.switch_to.window(window)
            self.browser.close()
        self.browser.switch_to.window(self.browser.window_handles[0])
        if len(self.post_count_dic) < self.max_hashtag:
            self.get_related(self.browser.current_url) 
        else:
            self.browser.quit()
            self.save_file()


    def start(self):
        self.login()
        self.get_related(f'https://www.instagram.com/explore/tags/{self.main_hashtag}')

    
    

In [35]:
Instaminer("dog", 20).start()



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\guswn\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


KeyboardInterrupt: 

In [43]:
insta_ID = config['insta_ID']
insta_PD = config['insta_PD']

## 검색, 인기게시물
main_hashtag = "dog"
max_hashtag = 15

post_count_dic = {}

In [36]:
def wait_for(locator):
    return WebDriverWait(browser, 4).until(
    EC.presence_of_element_located(locator)
    )

def login():
    
    browser.get(f'https://www.instagram.com/accounts/login/')
    
    wait_for((By.CLASS_NAME, "_2hvTZ"))

    insta_id = browser.find_element_by_name('username')
    insta_pw = browser.find_element_by_name('password')

    insta_id.send_keys(insta_ID)
    insta_pw.send_keys(insta_PD)
    insta_pw.send_keys(Keys.ENTER)

    time.sleep(3)

## 탭마다 해시태그와 게시물 수를 가져와 딕셔너리에 넣음, 탭닫기.
def extract_data():
    try:
        hashtag = wait_for((By.CLASS_NAME, '_7UhW9')).text[1:]
        posts_count = int(wait_for((By.CLASS_NAME, 'g47SY')).text.replace(',',''))
        if hashtag and posts_count:
            post_count_dic[hashtag] = posts_count
    except:
        pass
       
#     for window in browser.window_handles[:-1]:
#         browser.close()
#         browser.switch_to.window(window)
        

In [29]:
def get_related(search_url):
    browser.get(search_url)
    time.sleep(1)
    pop_tag_li = poppost2tag(browser, frequency=10)
    
    ## 많이 나온 태그들 다시 검색해서 새탭에서 열기
    for tag in pop_tag_li:
        hashtag_name = clean_hashtag(tag)
        if hashtag_name not in pop_tag_li:
            search_url = f'https://www.instagram.com/explore/tags/{tag}'
            browser.execute_script("window.open('" +search_url +"');")
    for i in browser.window_handles:
        browser.switch_to.window(i)    
        extract_data()
        time.sleep(1)
    
    if len(post_count_dic) < max_hashtag:
        for window in browser.window_handles[:-1]:
            browser.close()
            browser.switch_to.window(window)
        
        browser.switch_to.window(browser.window_handles[0])
        get_related(browser.current_url)
    

## 인기게시물 9개를 하나씩 켜서 태그들을 tag_list에 저장하는 함수.
def poppost2tag(browser, frequency):
    pop_posts = browser.find_elements_by_class_name('_9AhH0')[:9]
    pop_posts_tags = []

    for post in pop_posts:
        post.click()
        try:
            wait_for((By.CLASS_NAME, "xil3i"))
            tag_list = browser.find_elements_by_class_name('xil3i')
            for tag in tag_list:
                pop_posts_tags.append(tag.text[1:])
        except:
            pass
        ActionChains(browser).send_keys(Keys.ESCAPE).perform()
    ## 인기게시물들의 태그들 중 빈도수 순 10개 가져오기
#     pop_tags = Counter(pop_posts_tags).most_common(frequency)

    pop_tags = pop_posts_tags[:10]
#     pop_tag_li = [i for i,c in pop_tags]
#     pop_tag_li.remove(main_hashtag)
    return pop_tags

In [44]:
post_count_dic = {}
browser = webdriver.Chrome(ChromeDriverManager().install())
login()

get_related(f'https://www.instagram.com/explore/tags/{main_hashtag}')



INFO:WDM:

====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
Current google-chrome version is 91.0.4472
INFO:WDM:Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
INFO:WDM:Get LATEST driver version for 91.0.4472
Driver [C:\Users\guswn\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\guswn\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [55]:
with open(f"{main_hashtag}-report.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(['Hashtag', 'Post Count'])
    for k,v in post_count_dic.items():
        writer.writerow([k,v])


In [56]:
# pop_tags = ko.vocab().most_common(10)

# ko = nltk.Text(pop_posts_tags, name='태그')

# plt.figure(figsize=(12,6))
# ko.plot(50)

# wordcloud = WordCloud(relative_scaling=0.2, background_color='white',
#                      ).generate_from_frequencies(dict(data))

# plt.figure(figsize=(12,8))
# plt.imshow(wordcloud)

In [ ]:
# from wordcloud import WordCloud, STOPWORDS
# import matplotlib.pyplot as plt
# import nltk